In [13]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [14]:
# Load dataset
product_data = pd.read_csv('/content/product_data.csv')  
user_preferences = pd.read_csv('/content/user_preferences.csv')  


In [15]:
# Menggabungkan dataset produk dengan preferensi pengguna berdasarkan atribut yang relevan
merged_data = pd.merge(product_data, user_preferences, on='Product_ID')  

KeyError: ignored

In [4]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Product_ID           50 non-null     object
 1   Product_Name         50 non-null     object
 2   Product_Colour_x     50 non-null     object
 3   Product_Quality_x    50 non-null     object
 4   Product_Size_x       50 non-null     object
 5   Product_Thickness_x  50 non-null     object
 6   Product_Price        50 non-null     int64 
 7   Product_Image        50 non-null     object
 8   Customer_ID          50 non-null     object
 9   Product_Colour_y     50 non-null     object
 10  Product_Quality_y    50 non-null     object
 11  Product_Size_y       50 non-null     object
 12  Product_Thickness_y  50 non-null     object
dtypes: int64(1), object(12)
memory usage: 5.5+ KB


In [5]:
merged_data.head()

,Product_ID,Product_Name,Product_Colour_x,Product_Quality_x,Product_Size_x,Product_Thickness_x,Product_Price,Product_Image,Customer_ID,Product_Colour_y,Product_Quality_y,Product_Size_y,Product_Thickness_y
0,TS001,[Cotton Combed] Black T-Shirt,Black,Cotton Combed,S,20s,26000,https://drive.google.com/file/d/1vTjsa6cHpNKIA...,CS027,Black,Cotton Combed,S,20s
1,TS002,[Cotton Combed] Grey T-Shirt,Grey,Cotton Combed,S,20s,26000,https://drive.google.com/file/d/12VlmrpPKReSwf...,CS044,Grey,Cotton Combed,S,20s
2,TS008,[Polyester] Grey T-Shirt,Grey,Polyester,S,0,24000,https://drive.google.com/file/d/1LT0x19DHadkDd...,CS010,Grey,Polyester,S,0
3,TS008,[Polyester] Grey T-Shirt,Grey,Polyester,S,0,24000,https://drive.google.com/file/d/1LT0x19DHadkDd...,CS036,Grey,Polyester,S,0
4,TS011,[Polyester] Sage T-Shirt,Sage,Polyester,S,0,24000,https://drive.google.com/file/d/1aSSPSUeAhL5Bk...,CS019,Sage,Polyester,S,0


In [7]:
# merged_data.to_excel('dataset_join.xlsx')

In [6]:
# Menentukan atribut yang akan digunakan sebagai fitur
features = ['Product_Name', 'Product_Colour_y', 'Product_Quality_y', 'Product_Size_y', 'Product_Thickness_y']  


In [7]:
# Menggunakan LabelEncoder untuk mengubah atribut kategori menjadi nilai numerik
label_encoder = LabelEncoder()
for feature in features:
    merged_data[feature] = label_encoder.fit_transform(merged_data[feature])

In [8]:
merged_data.head()

,Product_ID,Product_Name,Product_Colour_x,Product_Quality_x,Product_Size_x,Product_Thickness_x,Product_Price,Product_Image,Customer_ID,Product_Colour_y,Product_Quality_y,Product_Size_y,Product_Thickness_y
0,TS001,0,Black,Cotton Combed,S,20s,26000,https://drive.google.com/file/d/1vTjsa6cHpNKIA...,CS027,0,0,2,1
1,TS002,1,Grey,Cotton Combed,S,20s,26000,https://drive.google.com/file/d/12VlmrpPKReSwf...,CS044,1,0,2,1
2,TS008,7,Grey,Polyester,S,0,24000,https://drive.google.com/file/d/1LT0x19DHadkDd...,CS010,1,1,2,0
3,TS008,7,Grey,Polyester,S,0,24000,https://drive.google.com/file/d/1LT0x19DHadkDd...,CS036,1,1,2,0
4,TS011,9,Sage,Polyester,S,0,24000,https://drive.google.com/file/d/1aSSPSUeAhL5Bk...,CS019,4,1,2,0


In [14]:
# merged_data.to_excel('dataset_join_afterencode.xlsx')

In [9]:
# Memisahkan fitur dan target
X = merged_data[features].values
y = merged_data['Product_Name'].values  

In [10]:
# Standarisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
# Split data menjadi data pelatihan dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Create TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  
])

In [19]:
# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
# Make predictions
predictions = model.predict(X_test)

In [ ]:
# Post-process predictions
recommended_products = []
for prediction in predictions:
    # Mengambil indeks dengan probabilitas prediksi tertinggi
    recommended_products.append(label_encoder.inverse_transform([prediction.argmax()])[0])

In [ ]:
# Menampilkan rekomendasi produk
print('Recommended Products:')
for product_id in recommended_products:
    product_name = product_data.loc[product_data['Product_ID'] == product_id, 'Product_Name'].values[0]  
    print(product_name)